In [111]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}

In [67]:
val train = spark.read
        .options(Map("inferSchema"->"true","delimiter"->",","header"->"true"))
        .csv("/labs/slaba05/lab05_train.csv")

train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [66]:
val test = spark.read
        .options(Map("inferSchema"->"true","delimiter"->",","header"->"true"))
        .csv("/labs/slaba05/lab05_test.csv")

test = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

TRAIN

In [8]:
train.printSchema

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [ ]:
train.show(numRows = 3, truncate = 100, vertical=true)

In [24]:
def countCols(columns:Array[String]):Array[Column]={
    columns.map(c=>{
      count(when(col(c).isNull || 
                 col(c)==="" || 
                 col(c).contains("NULL") || 
                 col(c).contains("null"),c)
           ).alias(c)
    })
}
train.select(countCols(train.columns):_*).show(50, 100, true)

countCols: (columns: Array[String])Array[org.apache.spark.sql.Column]


-RECORD 0-----------------------------
 _c0                         | 0      
 ID                          | 0      
 CR_PROD_CNT_IL              | 0      
 AMOUNT_RUB_CLO_PRC          | 34550  
 PRC_ACCEPTS_A_EMAIL_LINK    | 180814 
 APP_REGISTR_RGN_CODE        | 265914 
 PRC_ACCEPTS_A_POS           | 180814 
 PRC_ACCEPTS_A_TK            | 180814 
 TURNOVER_DYNAMIC_IL_1M      | 1      
 CNT_TRAN_AUT_TENDENCY1M     | 251229 
 SUM_TRAN_AUT_TENDENCY1M     | 251229 
 AMOUNT_RUB_SUP_PRC          | 34551  
 PRC_ACCEPTS_A_AMOBILE       | 180814 
 SUM_TRAN_AUT_TENDENCY3M     | 220560 
 CLNT_TRUST_RELATION         | 257829 
 PRC_ACCEPTS_TK              | 180814 
 PRC_ACCEPTS_A_MTP           | 180814 
 REST_DYNAMIC_FDEP_1M        | 1      
 CNT_TRAN_AUT_TENDENCY3M     | 220560 
 CNT_ACCEPTS_TK              | 180814 
 APP_MARITAL_STATUS          | 258989 
 REST_DYNAMIC_SAVE_3M        | 1      
 CR_PROD_CNT_VCU             | 1      
 REST_AVG_CUR                | 1      
 CNT_TRAN_MED_TENDENCY1M 

In [68]:
// удалим колонки с большим количеством пустых значений

val cols_to_drop = Seq(
    "PRC_ACCEPTS_A_EMAIL_LINK",
    "APP_REGISTR_RGN_CODE",
    "PRC_ACCEPTS_A_POS",
    "PRC_ACCEPTS_A_TK",
    "CNT_TRAN_AUT_TENDENCY1M",
    "SUM_TRAN_AUT_TENDENCY1M",
    "PRC_ACCEPTS_A_AMOBILE",
    "SUM_TRAN_AUT_TENDENCY3M",
    "CLNT_TRUST_RELATION",
    "PRC_ACCEPTS_TK",
    "PRC_ACCEPTS_A_MTP",
    "CNT_TRAN_AUT_TENDENCY3M",
    "CNT_ACCEPTS_TK",
    "APP_MARITAL_STATUS",
    "CNT_TRAN_MED_TENDENCY1M",
    "APP_KIND_OF_PROP_HABITATION",
    "CLNT_JOB_POSITION_TYPE",
    "APP_DRIVING_LICENSE",
    "APP_EDUCATION",
    "CNT_TRAN_CLO_TENDENCY1M",
    "SUM_TRAN_MED_TENDENCY1M",
    "PRC_ACCEPTS_A_ATM",
    "PRC_ACCEPTS_MTP",
    "APP_TRAVEL_PASS",
    "CNT_ACCEPTS_MTP",
    "APP_CAR",
    "SUM_TRAN_CLO_TENDENCY1M",
    "APP_POSITION_TYPE",
    "CNT_TRAN_MED_TENDENCY3M",
    "SUM_TRAN_MED_TENDENCY3M",
    "APP_EMP_TYPE",
    "SUM_TRAN_CLO_TENDENCY3M",
    "LDEAL_TENOR_MAX",
    "LDEAL_YQZ_CHRG",
    "DEAL_YQZ_IR_MAX",
    "LDEAL_YQZ_COM",
    "DEAL_YQZ_IR_MIN",
    "CNT_TRAN_CLO_TENDENCY3M",
    "LDEAL_TENOR_MIN",
    "LDEAL_AMT_MONTH",
    "APP_COMP_TYPE",
    "CNT_TRAN_SUP_TENDENCY3M",
    "SUM_TRAN_SUP_TENDENCY3M",
    "CNT_TRAN_ATM_TENDENCY1M",
    "SUM_TRAN_ATM_TENDENCY3M",
    "DEAL_GRACE_DAYS_ACC_S1X1",
    "AVG_PCT_MONTH_TO_PCLOSE",
    "DEAL_YWZ_IR_MIN",
    "SUM_TRAN_SUP_TENDENCY1M",
    "DEAL_YWZ_IR_MAX",
    "SUM_TRAN_ATM_TENDENCY1M",
    "CNT_TRAN_SUP_TENDENCY1M",
    "DEAL_GRACE_DAYS_ACC_AVG",
    "MAX_PCLOSE_DATE",
    "LDEAL_YQZ_PC",
    "DEAL_GRACE_DAYS_ACC_MAX",
    "LDEAL_DELINQ_PER_MAXYQZ",
    "CLNT_SALARY_VALUE",
    "TRANS_AMOUNT_TENDENCY3M",
    "MED_DEBT_PRC_YQZ",
    "LDEAL_USED_AMT_AVG_YQZ",
    "LDEAL_USED_AMT_AVG_YWZ",
    "AVG_PCT_DEBT_TO_DEAL_AMT",
    "LDEAL_ACT_DAYS_ACC_PCT_AVG",
    "MED_DEBT_PRC_YWZ",
    "LDEAL_ACT_DAYS_PCT_TR3",
    "LDEAL_ACT_DAYS_PCT_AAVG",
    "LDEAL_DELINQ_PER_MAXYWZ",
    "LDEAL_ACT_DAYS_PCT_TR",
    "LDEAL_ACT_DAYS_PCT_TR4",
    "LDEAL_ACT_DAYS_PCT_CURR",
    "CLNT_JOB_POSITION",
    "PACK"
)

cols_to_drop = List(PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, APP_TRAVEL_PASS, CNT_ACCEPTS_MTP, APP_CAR, SUM_TRAN_CLO_TENDENCY1M, APP_POSITION_TYPE, CNT_TRAN_MED_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, APP_EMP_TYPE, SUM_TRAN_CLO_TENDENCY3M, LDEAL_TENOR_MAX, LDEAL_YQZ_CHRG, DEAL_YQZ_IR_MAX, LDEAL_YQZ_COM, DEAL_YQZ_IR_MIN, CNT_TR...


List(PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, APP_TRAVEL_PASS, CNT_ACCEPTS_MTP, APP_CAR, SUM_TRAN_CLO_TENDENCY1M, APP_POSITION_TYPE, CNT_TRAN_MED_TENDENCY3M, SUM_TRAN_MED_TENDENCY3M, APP_EMP_TYPE, SUM_TRAN_CLO_TENDENCY3M, LDEAL_TENOR_MAX, LDEAL_YQZ_CHRG, DEAL_YQZ_IR_MAX, LDEAL_YQZ_COM, DEAL_YQZ_IR_MIN, CNT_TR...

In [69]:
val new_train = train.drop(cols_to_drop:_*)
new_train.select(countCols(new_train.columns):_*).show(50, 100, true)

-RECORD 0-------------------------
 _c0                      | 0     
 ID                       | 0     
 CR_PROD_CNT_IL           | 0     
 AMOUNT_RUB_CLO_PRC       | 34550 
 TURNOVER_DYNAMIC_IL_1M   | 1     
 AMOUNT_RUB_SUP_PRC       | 34551 
 REST_DYNAMIC_FDEP_1M     | 1     
 REST_DYNAMIC_SAVE_3M     | 1     
 CR_PROD_CNT_VCU          | 1     
 REST_AVG_CUR             | 1     
 AMOUNT_RUB_NAS_PRC       | 34551 
 TRANS_COUNT_SUP_PRC      | 34551 
 TRANS_COUNT_NAS_PRC      | 34551 
 CR_PROD_CNT_TOVR         | 1     
 CR_PROD_CNT_PIL          | 1     
 TURNOVER_CC              | 1     
 TRANS_COUNT_ATM_PRC      | 34551 
 AMOUNT_RUB_ATM_PRC       | 34551 
 TURNOVER_PAYM            | 1     
 AGE                      | 1     
 CR_PROD_CNT_CC           | 1     
 REST_DYNAMIC_FDEP_3M     | 1     
 REST_DYNAMIC_IL_1M       | 1     
 CR_PROD_CNT_CCFP         | 1     
 REST_DYNAMIC_CUR_1M      | 1     
 REST_AVG_PAYM            | 1     
 LDEAL_GRACE_DAYS_PCT_MED | 1     
 REST_DYNAMIC_CUR_3M

new_train = [_c0: int, ID: int ... 42 more fields]


[_c0: int, ID: int ... 42 more fields]

In [70]:
val fillData: Map[String, Any] = Map(
    "AMOUNT_RUB_CLO_PRC" -> 0,
    "AMOUNT_RUB_SUP_PRC" -> 0,
    "AMOUNT_RUB_NAS_PRC" -> 0,
    "TRANS_COUNT_SUP_PRC" -> 0,
    "TRANS_COUNT_NAS_PRC" -> 0,
    "TRANS_COUNT_ATM_PRC" -> 0,
    "AMOUNT_RUB_ATM_PRC" -> 0,
    "CNT_TRAN_ATM_TENDENCY3M" -> 0,
    "TRANS_CNT_TENDENCY3M" -> 0
)

val clean_train = new_train.na.fill(fillData).na.drop("any")

fillData = Map(AMOUNT_RUB_SUP_PRC -> 0, AMOUNT_RUB_CLO_PRC -> 0, AMOUNT_RUB_NAS_PRC -> 0, TRANS_COUNT_SUP_PRC -> 0, TRANS_CNT_TENDENCY3M -> 0, TRANS_COUNT_NAS_PRC -> 0, CNT_TRAN_ATM_TENDENCY3M -> 0, AMOUNT_RUB_ATM_PRC -> 0, TRANS_COUNT_ATM_PRC -> 0)
clean_train = [_c0: int, ID: int ... 42 more fields]


[_c0: int, ID: int ... 42 more fields]

In [71]:
clean_train.select(countCols(clean_train.columns):_*).show(50, 100, true)

-RECORD 0-----------------------
 _c0                      | 0   
 ID                       | 0   
 CR_PROD_CNT_IL           | 0   
 AMOUNT_RUB_CLO_PRC       | 0   
 TURNOVER_DYNAMIC_IL_1M   | 0   
 AMOUNT_RUB_SUP_PRC       | 0   
 REST_DYNAMIC_FDEP_1M     | 0   
 REST_DYNAMIC_SAVE_3M     | 0   
 CR_PROD_CNT_VCU          | 0   
 REST_AVG_CUR             | 0   
 AMOUNT_RUB_NAS_PRC       | 0   
 TRANS_COUNT_SUP_PRC      | 0   
 TRANS_COUNT_NAS_PRC      | 0   
 CR_PROD_CNT_TOVR         | 0   
 CR_PROD_CNT_PIL          | 0   
 TURNOVER_CC              | 0   
 TRANS_COUNT_ATM_PRC      | 0   
 AMOUNT_RUB_ATM_PRC       | 0   
 TURNOVER_PAYM            | 0   
 AGE                      | 0   
 CR_PROD_CNT_CC           | 0   
 REST_DYNAMIC_FDEP_3M     | 0   
 REST_DYNAMIC_IL_1M       | 0   
 CR_PROD_CNT_CCFP         | 0   
 REST_DYNAMIC_CUR_1M      | 0   
 REST_AVG_PAYM            | 0   
 LDEAL_GRACE_DAYS_PCT_MED | 0   
 REST_DYNAMIC_CUR_3M      | 0   
 TURNOVER_DYNAMIC_CUR_1M  | 0   
 REST_DYNA

In [72]:
clean_train.count - new_train.count

-1

TEST

In [ ]:
test.show(numRows = 3, truncate = 100, vertical=true)

In [74]:
test.select(countCols(test.columns):_*).show(50, 100, true)

-RECORD 0----------------------------
 _c0                         | 0     
 ID                          | 0     
 CR_PROD_CNT_IL              | 0     
 AMOUNT_RUB_CLO_PRC          | 4794  
 PRC_ACCEPTS_A_EMAIL_LINK    | 25021 
 APP_REGISTR_RGN_CODE        | 36874 
 PRC_ACCEPTS_A_POS           | 25021 
 PRC_ACCEPTS_A_TK            | 25021 
 TURNOVER_DYNAMIC_IL_1M      | 0     
 CNT_TRAN_AUT_TENDENCY1M     | 34732 
 SUM_TRAN_AUT_TENDENCY1M     | 34732 
 AMOUNT_RUB_SUP_PRC          | 4794  
 PRC_ACCEPTS_A_AMOBILE       | 25021 
 SUM_TRAN_AUT_TENDENCY3M     | 30628 
 CLNT_TRUST_RELATION         | 35857 
 PRC_ACCEPTS_TK              | 25021 
 PRC_ACCEPTS_A_MTP           | 25021 
 REST_DYNAMIC_FDEP_1M        | 0     
 CNT_TRAN_AUT_TENDENCY3M     | 30628 
 CNT_ACCEPTS_TK              | 25021 
 APP_MARITAL_STATUS          | 35930 
 REST_DYNAMIC_SAVE_3M        | 0     
 CR_PROD_CNT_VCU             | 0     
 REST_AVG_CUR                | 0     
 CNT_TRAN_MED_TENDENCY1M     | 35880 
 APP_KIND_OF

In [75]:
val new_test = test.drop(cols_to_drop:_*)
new_test.select(countCols(new_test.columns):_*).show(50, 100, true)

-RECORD 0-------------------------
 _c0                      | 0     
 ID                       | 0     
 CR_PROD_CNT_IL           | 0     
 AMOUNT_RUB_CLO_PRC       | 4794  
 TURNOVER_DYNAMIC_IL_1M   | 0     
 AMOUNT_RUB_SUP_PRC       | 4794  
 REST_DYNAMIC_FDEP_1M     | 0     
 REST_DYNAMIC_SAVE_3M     | 0     
 CR_PROD_CNT_VCU          | 0     
 REST_AVG_CUR             | 0     
 AMOUNT_RUB_NAS_PRC       | 4794  
 TRANS_COUNT_SUP_PRC      | 4794  
 TRANS_COUNT_NAS_PRC      | 4794  
 CR_PROD_CNT_TOVR         | 0     
 CR_PROD_CNT_PIL          | 0     
 TURNOVER_CC              | 0     
 TRANS_COUNT_ATM_PRC      | 4794  
 AMOUNT_RUB_ATM_PRC       | 4794  
 TURNOVER_PAYM            | 0     
 AGE                      | 0     
 CR_PROD_CNT_CC           | 0     
 REST_DYNAMIC_FDEP_3M     | 0     
 REST_DYNAMIC_IL_1M       | 0     
 CR_PROD_CNT_CCFP         | 0     
 REST_DYNAMIC_CUR_1M      | 0     
 REST_AVG_PAYM            | 0     
 LDEAL_GRACE_DAYS_PCT_MED | 0     
 REST_DYNAMIC_CUR_3M

new_test = [_c0: int, ID: int ... 41 more fields]


[_c0: int, ID: int ... 41 more fields]

In [76]:
val clean_test = new_test.na.fill(fillData).na.drop("any")

clean_test = [_c0: int, ID: int ... 41 more fields]


[_c0: int, ID: int ... 41 more fields]

In [77]:
clean_test.select(countCols(clean_test.columns):_*).show(50, 100, true)

-RECORD 0-----------------------
 _c0                      | 0   
 ID                       | 0   
 CR_PROD_CNT_IL           | 0   
 AMOUNT_RUB_CLO_PRC       | 0   
 TURNOVER_DYNAMIC_IL_1M   | 0   
 AMOUNT_RUB_SUP_PRC       | 0   
 REST_DYNAMIC_FDEP_1M     | 0   
 REST_DYNAMIC_SAVE_3M     | 0   
 CR_PROD_CNT_VCU          | 0   
 REST_AVG_CUR             | 0   
 AMOUNT_RUB_NAS_PRC       | 0   
 TRANS_COUNT_SUP_PRC      | 0   
 TRANS_COUNT_NAS_PRC      | 0   
 CR_PROD_CNT_TOVR         | 0   
 CR_PROD_CNT_PIL          | 0   
 TURNOVER_CC              | 0   
 TRANS_COUNT_ATM_PRC      | 0   
 AMOUNT_RUB_ATM_PRC       | 0   
 TURNOVER_PAYM            | 0   
 AGE                      | 0   
 CR_PROD_CNT_CC           | 0   
 REST_DYNAMIC_FDEP_3M     | 0   
 REST_DYNAMIC_IL_1M       | 0   
 CR_PROD_CNT_CCFP         | 0   
 REST_DYNAMIC_CUR_1M      | 0   
 REST_AVG_PAYM            | 0   
 LDEAL_GRACE_DAYS_PCT_MED | 0   
 REST_DYNAMIC_CUR_3M      | 0   
 TURNOVER_DYNAMIC_CUR_1M  | 0   
 REST_DYNA

In [ ]:
val train_columns_to_select = clean_train.columns.drop(1)
val train_columns_to_features = clean_train.columns.drop(2).dropRight(1)

In [96]:
val assembler = new VectorAssembler().setInputCols(train_columns_to_features).setOutputCol("features")
val gbt = new GBTClassifier().setLabelCol("TARGET").setFeaturesCol("features").setMaxDepth(7).setFeatureSubsetStrategy("auto")

assembler = vecAssembler_492996605198
gbt = gbtc_30af29039887


gbtc_30af29039887

In [98]:
val pipeline = new Pipeline().setStages(Array(assembler, gbt))

pipeline = pipeline_f30433a816be


pipeline_f30433a816be

In [100]:
val model = pipeline.fit(clean_train)

model = pipeline_f30433a816be


pipeline_f30433a816be

In [101]:
val predictions = model.transform(clean_test)

predictions = [_c0: int, ID: int ... 45 more fields]


[_c0: int, ID: int ... 45 more fields]

In [103]:
predictions.groupBy('prediction).count.show

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|44269|
|       1.0|  130|
+----------+-----+



In [112]:
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)

toArr = > Array[Double] = <function1>
toArrUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)
dataWithFeaturesArr = [_c0: int, ID: int ... 46 more fields]


[_c0: int, ID: int ... 46 more fields]

In [130]:
val result = predictions.select('ID, 'probability)
    .withColumn("probability_arr", toArrUdf('probability))
    .select('ID.alias("id"), element_at('probability_arr, 2).alias("target"))

result = [id: int, target: double]


[id: int, target: double]

In [134]:
result.show(10, 100, true)

-RECORD 0----------------------
 id     | 519130               
 target | 0.2556870669138509   
-RECORD 1----------------------
 id     | 234045               
 target | 0.04926471007562305  
-RECORD 2----------------------
 id     | 401256               
 target | 0.04814686291447734  
-RECORD 3----------------------
 id     | 551070               
 target | 0.047398589333296104 
-RECORD 4----------------------
 id     | 367285               
 target | 0.045669040159410645 
-RECORD 5----------------------
 id     | 497998               
 target | 0.05124033383249449  
-RECORD 6----------------------
 id     | 413082               
 target | 0.08172089909659153  
-RECORD 7----------------------
 id     | 349893               
 target | 0.045669040159410645 
-RECORD 8----------------------
 id     | 346337               
 target | 0.1455097315993732   
-RECORD 9----------------------
 id     | 289979               
 target | 0.04576391013280712  
only showing top 10 rows



In [133]:
result.repartition(1).write.option("header", true).option("delimiter", "\t").csv("lab05_3.csv")